# Student Loan Risk with Deep Learning

In [41]:
# Imports
import pandas as pd
import numpy as np
import io
import requests
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student_loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [42]:
# Read the csv into a Pandas DataFrame
url = "https://static.bc-edx.com/mbc/ai/m6/datasets/student_loans.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Review the DataFrame
df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [43]:
# Review the data types associated with the columns
df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [44]:
# Define the target set y using the credit_ranking column
y = df["credit_ranking"]

# Display a sample of y
y[:5]

0    5
1    5
2    5
3    6
4    5
Name: credit_ranking, dtype: int64

In [45]:
# Define features set X by selecting all columns but credit_ranking
X = df.copy()
X = X.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [46]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [47]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [48]:
# Define the the number of inputs (features) to the model
n_inputs=11

# Review the number of features
print(n_inputs)

11


In [49]:
# Define the number of neurons in the output layer
n_output=1

In [50]:
# Define the number of hidden nodes for the first hidden layer
n_nodes1=6

# Review the number hidden nodes in the first layer
print(n_nodes1)

6


In [51]:
# Define the number of hidden nodes for the second hidden layer
n_nodes2=3

# Review the number hidden nodes in the second layer
print(n_nodes2)

3


In [52]:
# Create the Sequential model instance
nn = Sequential()

In [53]:
# Add the first hidden layer
nn.add(Dense(units=n_nodes1, input_dim=n_inputs, activation='relu'))

In [54]:
# Add the second hidden layer
nn.add(Dense(units=n_nodes2, activation='relu'))

In [55]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=n_output, activation='linear'))

In [56]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `mse` loss function, the `adam` optimizer, and the `mse` evaluation metric.


In [57]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [58]:
# Fit the model using 50 epochs and the training data
nn_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
 1/38 [..............................] - ETA: 10s - loss: 36.8794 - mse: 36.8794

2023-09-26 20:02:22.556278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 0s 4ms/step - loss: 33.1299 - mse: 33.1299
Epoch 2/50
38/38 [==============================] - 0s 4ms/step - loss: 30.7337 - mse: 30.7337
Epoch 3/50
38/38 [==============================] - 0s 4ms/step - loss: 29.0428 - mse: 29.0428
Epoch 4/50
38/38 [==============================] - 0s 4ms/step - loss: 27.6779 - mse: 27.6779
Epoch 5/50
38/38 [==============================] - 0s 4ms/step - loss: 26.5344 - mse: 26.5344
Epoch 6/50
38/38 [==============================] - 0s 4ms/step - loss: 25.5293 - mse: 25.5293
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 24.5556 - mse: 24.5556
Epoch 8/50
38/38 [==============================] - 0s 4ms/step - loss: 23.5466 - mse: 23.5466
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 22.5682 - mse: 22.5682
Epoch 10/50
38/38 [==============================] - 0s 4ms/step - loss: 21.5555 - mse: 21.5555
Epoch 11/50
38/38 [==============================] - 0s 4ms/

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [59]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_mse = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f'Loss: {model_loss}, Accuracy: {model_mse}')

13/13 - 0s - loss: 0.4249 - mse: 0.4249 - 95ms/epoch - 7ms/step
Loss: 0.42486798763275146, Accuracy: 0.42486798763275146


2023-09-26 20:02:33.704551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


### Step 4: Save and export your model to an HDF5 file, and name the file `student_loans.h5`.


In [60]:
# Set the model's file path
file_path = Path("saved_models/student_loans.h5")

# Export your model to a HDF5 file
nn.save(file_path)

/Users/mhaynam/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [61]:
# Set the model's file path
file_path = Path("saved_models/student_loans.h5")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data.

In [62]:
# Make predictions on the testing data
predictions = nn_imported.predict(X_test_scaled).round().astype("int32")

13/13 [==============================] - 0s 2ms/step


2023-09-26 20:02:43.518018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


### Step 3: Create a DataFrame to compare the predictions with the actual values.

In [63]:
# Create a DataFrame to compare the predictions with the actual values
results = pd.DataFrame({"predictions": predictions.ravel(), "actual": y_test})

### Step 4: Display a sample of the DataFrame you created in step 3.

In [66]:
# Display sample data
results.head(10)

,predictions,actual
75,6,5
1283,5,6
408,6,6
1281,6,6
1118,7,6
1143,6,6
1215,6,6
181,5,5
1186,5,5
1252,5,5


In [ ]:
7 out of the first 10 predictions are correct and the other 3 are only off by one. We could attempt to improve the model by playing around with a different number of layers and a different number of nodes.